In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt 
import mpmath
import timeit

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("./ZERN/"))

from zern.zern_core import Zernike
import numpy as np

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("./zernike/"))
from zernike import RZern

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("./zernpy/src/"))
from zernpy import ZernPol

from zernipax.basis import ZernikePolynomial
from zernipax.zernike import *
from zernipax.plotting import plot_comparison
from zernipax.backend import jax
from tqdm import tqdm

using JAX backend, jax version=0.4.28, jaxlib version=0.4.28, dtype=float64
Using device: CPU, with 10.28 GB available memory


In [ ]:
res = 50
basis = ZernikePolynomial(L=res, M=res, spectral_indexing="ansi", sym="cos")
ms = basis.modes[:,1]
ns = basis.modes[:,0]
r = np.linspace(0, 1, 100)

In [ ]:
def fun_zernipax_clear(ns, ms, r):
    jax.clear_caches()
    return zernike_radial(r, ns, ms, 0)

def fun_zernipax(ns, ms, r):
    return zernike_radial(r, ns, ms, 0)

# %timeit fun_zernipax(ns, ms, r).block_until_ready()
# %timeit fun_zernipax_clear(ns, ms, r).block_until_ready()
all_zernipax = fun_zernipax(ns, ms, r)

In [ ]:
def fun_zern(ns, ms, r):
    zern = Zernike(0)
    all = []
    for i in range(len(ms)): 
        all.append(zern.R_nm_Jacobi(int(ns[i]), int(ms[i]), r))

    return np.array(all)

all_zern = fun_zern(ns, ms, r).T

In [ ]:
def get_Noll(n, m):
    j = n*(n+1)//2 + abs(m)
    if m >= 0 and (n%4 == 2 or n%4 == 3):
        j += 1
    elif m <= 0 and (n%4 == 0 or n%4 == 1):
        j += 1
    return j

def fun_zernike(ns, ms, r):
    all = []
    cart = RZern(int(max(ns)))
    for i in range(len(ms)):
        id_Noll = get_Noll(ns[i], ms[i]) - 1
        all.append(cart.Rnm(id_Noll, r))

    return np.array(all)

all_zernike = fun_zernike(ns, ms, r).T

In [ ]:
def fun_zernpy(ns, ms, r):
    all = []
    for i in range(len(ms)):
        zp = ZernPol(m=int(ms[i]), n=int(ns[i]))   
        all.append(zp.radial(r))

    return np.array(all)

all_zernpy = fun_zernpy(ns, ms, r).T

In [ ]:
# Exact computation
mpmath.mp.dps = 100
def fun_exact(ns, ms, r):
    c = zernike_radial_coeffs(ns, ms, exact=True)
    zt0 = np.array([np.asarray(mpmath.polyval(list(ci), r), dtype=float) for ci in c]).T
    return zt0

exact = fun_exact(ns, ms, r)

In [ ]:
plot_comparison(
    exact, 
    (all_zernipax, all_zern),
    basis, 
    dx=0, 
    type="absolute", 
    names=("Zernipax:", "Zern:"), 
    print_error=True
)

In [ ]:
plot_comparison(
    exact, 
    (all_zernipax, all_zern, all_zernike, all_zernpy),
    basis, 
    dx=0, 
    type="absolute", 
    names=("Zernipax:", "Zern:", "Zernike:", "Zernpy:"), 
    print_error=True
)
plt.savefig("comparison_all.png", dpi=1000)

In [ ]:
print("Time for ZERN:")
%timeit fun_zern(ns, ms, r);
print("Time for ZERNIKE:")
%timeit fun_zernike(ns, ms, r);
# print("Time for ZERNPY:")
# %timeit fun_zernpy(ns, ms, r);
print("Time for ZERNIPAX:")
%timeit fun_zernipax(ns, ms, r).block_until_ready();

In [ ]:
r = np.linspace(0, 1, 100)
times = []
num_exec = 100
range_res = np.arange(10, 101, 5)

for res in tqdm(range_res):
    basis = ZernikePolynomial(L=res, M=res, spectral_indexing="ansi", sym="cos")
    ms = basis.modes[:,1]
    ns = basis.modes[:,0]
    _ = fun_zernipax(ns, ms, r) # run to compile it once

    t1 = timeit.timeit(lambda: fun_zern(ns, ms, r), number=num_exec)
    t2 = timeit.timeit(lambda: fun_zernike(ns, ms, r), number=num_exec)
    t3 = timeit.timeit(lambda: fun_zernipax(ns, ms, r).block_until_ready(), number=num_exec)
    t4 = timeit.timeit(lambda: fun_zernipax_clear(ns, ms, r).block_until_ready(), number=num_exec)

    times.append([t1, t2, t3, t4])
times = np.array(times)*1000/num_exec

In [ ]:
plt.plot(range_res, times[:,0], label="ZERN")
plt.plot(range_res, times[:,1], label="ZERNIKE")
plt.plot(range_res, times[:,2], label="ZERNIPAX")
plt.plot(range_res, times[:,3], label="ZERNIPAX-Compile")
# plt.plot(range_res, 0.001*range_res**3, label="order")
plt.xlabel("Resolution")
plt.ylabel("Time (ms)")
plt.title("Time Comparison of Computation of Radial Zernike Polynomials")
plt.grid()
plt.legend()
plt.savefig("time_comparison_compile.pdf", dpi=1000)

In [ ]:
plt.plot(range_res, times[:,2], label="ZERNIPAX")
# plt.plot(range_res, 0.00003*range_res**3, label="order")
plt.xlabel("Resolution")
plt.ylabel("Time (ms)")
plt.title("Computation Time of Radial Zernike Polynomials in ZERNIPAX")
plt.grid()
plt.legend()
plt.savefig("time_zernipax.pdf", dpi=1000)

In [2]:
# Exact computation
def fun_exact_prec(ns, ms, r, prec):
    mpmath.mp.dps = prec 
    c = zernike_radial_coeffs(ns, ms, exact=True)
    zt0 = np.array([np.asarray(mpmath.polyval(list(ci), r), dtype=float) for ci in c]).T
    return zt0

In [3]:
range_prec = np.arange(20, 41, 5)
res = 100
basis = ZernikePolynomial(L=res, M=res, spectral_indexing="ansi", sym="cos")
ms = basis.modes[:,1]
ns = basis.modes[:,0]
r = np.linspace(0, 1, 100)
# diff = []
# val_old = fun_exact_prec(ns, ms, r, 100)
# for prec in tqdm(range_prec):
#     val = fun_exact_prec(ns, ms, r, prec)
#     diff.append(np.max(val - val_old))
# diff = np.array(diff)

In [ ]:
plt.plot(range_prec, diff)

In [41]:
def fun_random(prec):
    mpmath.mp.dps = prec
    a = mpmath.mpf('1e-50')
    b = mpmath.mpf('5.2')
    return mpmath.fsub(b, a)

print(fun_random(200))
print(5.2-1e-50)

5.19999999999999999999999999999999999999999999999999
5.2


In [45]:
mpmath.mp.dps = 8
print("Decimal places of precision (dps):", mpmath.mp.dps)
print("Bit precision (prec):", mpmath.mp.prec)

Decimal places of precision (dps): 8
Bit precision (prec): 30


In [42]:
array = np.array([[1.5, 2.5, 3.5], [4.5, 5.5, 6.5]])

# Save the array to a text file
np.savetxt('my_array.txt', array)

# Load the array from the text file
loaded_array = np.loadtxt('my_array.txt')

In [43]:
print(loaded_array)

[[1.5 2.5 3.5]
 [4.5 5.5 6.5]]
